In [ ]:
!pip install snowflake-connector-python pandas

In [ ]:
import snowflake.connector

In [ ]:
conn = snowflake.connector.connect(
    user='ABDELRAHMAN1ZAID1',
    password='!6z:8MUULDpAXi!',
    account='DRTSSDW-CB27875',  # e.g. 'xy12345.east-us-2.azure'
    warehouse='COMPUTE_WH',
    database='FOREX_RATES',
    schema='PUBLIC'
)

In [ ]:
import pandas as pd

In [ ]:
query = "SHOW TABLES"
tables_df = pd.read_sql(query, conn)
print(tables_df[['name', 'database_name', 'schema_name']])

In [ ]:
import pandas as pd

# Example: loading "FOREX_EUR_USD" from the "FOREX_DATA" database and "BRONZE" schema
query = "SELECT * FROM FOREX_DATA.SILVER.FOREX_RATES"
df = pd.read_sql(query, conn)


# Preview the data
print(df.head())


In [ ]:
df

In [ ]:
df = df.rename(columns={'OPEN_RATE': 'Open', 'HIGH_RATE': 'High','LOW_RATE':'Low','CLOSE_RATE':'Close'})

In [ ]:
df

# Machine Learning Model

In [ ]:
!pip install pandas numpy ta lightgbm plotly scikit-learn

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import ta
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import lightgbm as lgb

In [ ]:
# Step 1: Load Data
df['DATE'] = pd.to_datetime(df['DATE'])
df.sort_values('DATE', inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
# Step 2: Feature Engineering
df['HL'] = df['High'] - df['Low']
df['OC'] = df['Close'] - df['Open']
df['BODY'] = abs(df['Close'] - df['Open'])

In [ ]:
# Technical indicators
df['RSI'] = ta.momentum.RSIIndicator(df['Close'], window=14).rsi()
df['EMA_12'] = ta.trend.ema_indicator(df['Close'], window=12)
df['EMA_26'] = ta.trend.ema_indicator(df['Close'], window=26)
df['MACD'] = df['EMA_12'] - df['EMA_26']
bb = ta.volatility.BollingerBands(df['Close'])
df['BB_HIGH'] = bb.bollinger_hband()
df['BB_LOW'] = bb.bollinger_lband()
df['STOCH'] = ta.momentum.StochasticOscillator(df['High'], df['Low'], df['Close']).stoch()


In [ ]:
# Rolling features
for window in [3, 5, 10]:
    df[f'CLOSE_ROLL_MEAN_{window}'] = df['Close'].rolling(window).mean()
    df[f'CLOSE_ROLL_STD_{window}'] = df['Close'].rolling(window).std()
    df[f'HL_ROLL_MEAN_{window}'] = df['HL'].rolling(window).mean()
    df[f'OC_ROLL_MEAN_{window}'] = df['OC'].rolling(window).mean()

In [ ]:
# Time features
df['HOUR'] = df['DATE'].dt.hour
df['DAY_OF_WEEK'] = df['DATE'].dt.dayofweek

In [ ]:
# -----------------------------
# Step 3: Candlestick Pattern Features
# -----------------------------
df['IS_BULLISH'] = (df['Close'] > df['Open']).astype(int)
df['IS_BEARISH'] = (df['Close'] < df['Open']).astype(int)
df['IS_DOJI'] = (abs(df['Close'] - df['Open']) < (df['High'] - df['Low']) * 0.1).astype(int)

df['BODY'] = abs(df['Close'] - df['Open'])

df['IS_HAMMER'] = ((df['High'] - df['Low']) > 3 * df['BODY']) & \
                  ((df['Close'] - df['Low']) / (df['High'] - df['Low'] + 1e-9) > 0.6) & \
                  ((df['Open'] - df['Low']) / (df['High'] - df['Low'] + 1e-9) > 0.6)
df['IS_HAMMER'] = df['IS_HAMMER'].astype(int)

df['IS_INV_HAMMER'] = ((df['High'] - df['Low']) > 3 * df['BODY']) & \
                      ((df['High'] - df['Close']) / (df['High'] - df['Low'] + 1e-9) > 0.6) & \
                      ((df['High'] - df['Open']) / (df['High'] - df['Low'] + 1e-9) > 0.6)
df['IS_INV_HAMMER'] = df['IS_INV_HAMMER'].astype(int)

df['PREV_OPEN'] = df['Open'].shift(1)
df['PREV_CLOSE'] = df['Close'].shift(1)

df['BULLISH_ENGULFING'] = ((df['IS_BULLISH'] == 1) &
                           (df['PREV_CLOSE'] < df['PREV_OPEN']) &
                           (df['Open'] < df['PREV_CLOSE']) &
                           (df['Close'] > df['PREV_OPEN'])).astype(int)

df['BEARISH_ENGULFING'] = ((df['IS_BEARISH'] == 1) &
                           (df['PREV_CLOSE'] > df['PREV_OPEN']) &
                           (df['Open'] > df['PREV_CLOSE']) &
                           (df['Close'] < df['PREV_OPEN'])).astype(int)




In [ ]:
df

In [ ]:
import plotly.graph_objects as go

# Step 6: Visualization
# -----------------------------
plot_df = df[-200:].copy()

fig = go.Figure(data=[
    go.Candlestick(
        x=plot_df['DATE'],
        open=plot_df['Open'],  # Updated column names
        high=plot_df['High'],  # Updated column names
        low=plot_df['Low'],    # Updated column names
        close=plot_df['Close'], # Updated column names
        name="OHLC"
    )
])

# Add markers for patterns
def add_marker(condition, name, color, y_offset=0):
    dates = plot_df[condition]['DATE']
    prices = plot_df[condition]['High'] + y_offset  # Use High for marker placement
    fig.add_trace(go.Scatter(
        x=dates,
        y=prices,
        mode='markers',
        marker=dict(size=8, color=color),
        name=name
    ))

add_marker(plot_df['IS_DOJI'] == 1, 'Doji', 'orange')
add_marker(plot_df['IS_HAMMER'] == 1, 'Hammer', 'green')
add_marker(plot_df['IS_INV_HAMMER'] == 1, 'Inv Hammer', 'blue')
add_marker(plot_df['BULLISH_ENGULFING'] == 1, 'Bull Engulfing', 'lime')
add_marker(plot_df['BEARISH_ENGULFING'] == 1, 'Bear Engulfing', 'red')

fig.update_layout(
    title='Forex Candlestick Patterns with ML Features',
    xaxis_title='Date',
    yaxis_title='Price',
    xaxis_rangeslider_visible=True,  # Enabling range slider for time filtering
    xaxis_rangeslider_thickness=0.1,  # Adjust thickness of the slider
    xaxis=dict(
        rangeslider=dict(visible=True),
        type='date'  # Ensure date type for x-axis
    ),
    height=700
)

fig.show()

In [ ]:
# Drop non-numeric and irrelevant columns before training
exclude_cols = ['DATE', 'TARGET', 'CURRENCY_PAIR'] if 'CURRENCY_PAIR' in df.columns else ['DATE', 'TARGET']
features = [col for col in df.columns if col not in exclude_cols]


In [ ]:
# -----------------------------
# Step 4: Create Target Variable
# -----------------------------
df['TARGET'] = (df['Close'].shift(-1) > df['Close']).astype(int)
df.dropna(inplace=True)

In [ ]:
# Drop rows with NaNs (just in case)
df.dropna(inplace=True)

# Remove DATE and other non-numeric columns explicitly
non_numeric_cols = df.select_dtypes(include=['datetime64[ns]', 'object']).columns.tolist()
non_features = non_numeric_cols + ['TARGET']
features = [col for col in df.columns if col not in non_features]

# Confirm all feature columns are numeric
X = df[features].select_dtypes(include=[np.number])
y = df['TARGET']




In [ ]:
# Split data
split_idx = int(len(df) * 0.8)
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]



In [ ]:
# Time-based split
split_idx = int(len(df) * 0.8)
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

In [ ]:
# Train LightGBM model
model = lgb.LGBMClassifier(n_estimators=100, learning_rate=0.05)
model.fit(X_train, y_train)

In [ ]:
# Evaluation
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Model Test Accuracy: {acc:.2%}")

In [ ]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

model = lgb.LGBMClassifier(n_estimators=100, learning_rate=0.05)
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
plt.title("Confusion Matrix")
plt.show()

#Evaluating Other Models

In [ ]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Scale data for Logistic Regression
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Step 2: Train Multiple Models
# -----------------------------
models = {
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'XGBoost': XGBClassifier(n_estimators=100, learning_rate=0.05, use_label_encoder=False, eval_metric='logloss'),
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'LightGBM': lgb.LGBMClassifier(n_estimators=100, learning_rate=0.05),
    'Voting (all)': VotingClassifier(estimators=[
        ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
        ('xgb', XGBClassifier(n_estimators=100, learning_rate=0.05, use_label_encoder=False, eval_metric='logloss')),
        ('lr', LogisticRegression(max_iter=1000)),
    ], voting='hard')
}

for name, model in models.items():
    print(f"\nTraining {name}")
    if name == 'Logistic Regression':
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

    print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(classification_report(y_test, y_pred))
    sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
    plt.title(f"{name} - Confusion Matrix")
    plt.show()

# The Best Model Is Logistic Regression

In [ ]:
model=LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.01, 0.1, 1, 10, 100]}
grid_search = GridSearchCV(LogisticRegression(max_iter=1000), param_grid, cv=5)
grid_search.fit(X_train_scaled, y_train)

print("Best hyperparameters:", grid_search.best_params_)


In [ ]:
model=LogisticRegression(max_iter=1000,C =100)
model.fit(X_train_scaled, y_train)

In [ ]:
X_scaled = scaler.fit_transform(X)

In [ ]:
from sklearn.model_selection import cross_val_score

cv_scores = cross_val_score(model, X_scaled, y, cv=5)
print("Cross-validation scores:", cv_scores)

In [ ]:
predictions = model.predict(X_test_scaled)

In [ ]:
predictions[:5]

---> This Predict Candles Direction Up Or Down

#Model Deployment

In [ ]:
!pip install streamlit pyngrok joblib

In [ ]:
import joblib

# Save the model to a file
joblib.dump(model, 'logistic_regression_model.pkl')

In [ ]:
%%writefile app.py
import streamlit as st
import joblib
import numpy as np

# Load the trained model
model = joblib.load('logistic_regression_model.pkl')

# Title of the web app
st.title('Logistic Regression Model Prediction')

# Add some text input for the user to enter data
st.write("Enter features for prediction:")

# Define input fields for the features
feature1 = st.number_input('Feature 1', value=0.5)
feature2 = st.number_input('Feature 2', value=1.2)
feature3 = st.number_input('Feature 3', value=3.4)
feature4 = st.number_input('Feature 4', value=0.6)

# Prepare input for the model
features = np.array([[feature1, feature2, feature3, feature4]])

# Predict the label when the user clicks the button
if st.button('Make Prediction'):
    prediction = model.predict(features)
    st.write(f'The predicted label is: {prediction[0]}')


In [ ]:
!ngrok authtoken 2vxwvf2Iffxoxn3jRlT1i1ATQMV_2MrsQGhiMnNECHz8rfePK

In [ ]:
from pyngrok import ngrok

# Start the ngrok tunnel for streamlit
public_url = ngrok.connect(port='8501', proto='http')
print(f"Streamlit app is live at: {public_url}")

In [ ]:
!streamlit run app.py
